In [ ]:
import uproot
import awkward as ak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, DelphesSchema
from coffea.analysis_tools import Weights, PackedSelection
from coffea import hist, processor
from yahist import Hist1D, Hist2D

In [ ]:
from tools.helpers import *
from tools.basic_objects import getJets, getFatjets

In [ ]:
from tools.jes import JES

In [ ]:
jes_corrector = JES(verbose=False)

In [ ]:
events = NanoEventsFactory.from_root(
    '/hadoop/cms/store/user/ewallace/ProjectMetis/2HDMa_gg_1900_250_10_HLLHC_GEN_v6/delphes/delphes_3.root',
    treepath='Delphes',
    schemaclass = DelphesSchema,
).events()

In [ ]:
(events.Weight.Weight[:,423]/events.Weight.Weight[:,362]).tolist()

In [ ]:
events = NanoEventsFactory.from_root(
    'root://cmseos.fnal.gov//store/user/snowmass/Snowmass2021//Delphes/WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU/WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_43_2.root',
    treepath='Delphes',
    schemaclass = DelphesSchema,
).events()

In [ ]:
from yaml import Loader, Dumper
import yaml
with open('../data/samples.yaml', 'r') as f:
    samples = yaml.load(f, Loader = Loader)

fileset = {
'WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['delphes']
}

In [ ]:
import os

run2_to_delphes = {
    'WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': 'WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8',
}

effs = {}
for s in fileset.keys():
    effs[s] = {}
    for b in ['0b', '1b', '2b', '1h']:
        effs[s][b] = Hist2D.from_json(os.path.expandvars("../data/htag/eff_%s_%s.json"%(run2_to_delphes[s],b)))

In [ ]:
events.Weight.Weight

In [ ]:
for file in fileset['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']:
    events = NanoEventsFactory.from_root(
        file,
        treepath='Delphes',
        schemaclass = DelphesSchema,
    ).events()
    events = events[ak.num(events.Weight.Weight)!=0]
    if len(events) == 0:
        print(file)

In [ ]:
events = events[ak.num(events.Weight.Weight)!=0]
#define objects
        
#electrons
ele_sel_l = (events.ElectronLoose.PT>10)
electron_l = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.ElectronLoose.PT[ele_sel_l],
    eta = events.ElectronLoose.Eta[ele_sel_l],
    phi = events.ElectronLoose.Phi[ele_sel_l],
    M = np.zeros_like(events.ElectronLoose.PT[ele_sel_l]),
    copy = False,
)
        
electron_l['iso'] = events.ElectronLoose.IsolationVar[ele_sel_l]
ele_l = electron_l[((electron_l['iso']<0.3)&(electron_l.pt>10)&(np.abs(electron_l.eta)<3))]
        
ele_sel_m = (events.ElectronMedium.PT>10)
electron_m = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.ElectronMedium.PT[ele_sel_m],
    eta = events.ElectronMedium.Eta[ele_sel_m],
    phi = events.ElectronMedium.Phi[ele_sel_m],
    M = np.zeros_like(events.ElectronMedium.PT[ele_sel_m]),
    copy = False,
)
        
electron_m['iso'] = events.ElectronMedium.IsolationVar[ele_sel_m]

ele_m = electron_m[((electron_m['iso']<0.3)&(electron_m.pt>10)&(np.abs(electron_m.eta)<3))]
        
ele_sel_t = (events.ElectronTight.PT>10)
electron_t = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.ElectronTight.PT[ele_sel_t],
    eta = events.ElectronTight.Eta[ele_sel_t],
    phi = events.ElectronTight.Phi[ele_sel_t],
    M = np.zeros_like(events.ElectronTight.PT[ele_sel_t]),
    copy = False,
)
        
electron_t['iso'] = events.ElectronTight.IsolationVar[ele_sel_t]

ele_t = electron_t[((electron_t['iso']<0.3)&(electron_t.pt>10)&(np.abs(electron_t.eta)<3))]

#muons
mu_sel_l = (events.MuonLoose.PT>4)
muon_l = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.MuonLoose.PT[mu_sel_l],
    eta = events.MuonLoose.Eta[mu_sel_l],
    phi = events.MuonLoose.Phi[mu_sel_l],
    M = np.zeros_like(events.MuonLoose.PT[mu_sel_l]),
    copy = False,
)
muon_l['iso'] = events.MuonLoose.IsolationVar[mu_sel_l]

muon_l = muon_l[((muon_l['iso']<0.25)&(muon_l.pt>4)&(np.abs(muon_l.eta)<2.8))]
        
mu_sel_m = (events.MuonMedium.PT>4)
muon_m = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.MuonMedium.PT[mu_sel_m],
    eta = events.MuonMedium.Eta[mu_sel_m],
    phi = events.MuonMedium.Phi[mu_sel_m],
    M = np.zeros_like(events.MuonMedium.PT[mu_sel_m]),
    copy = False,
)
muon_m['iso'] = events.MuonMedium.IsolationVar[mu_sel_m]

muon_m = muon_m[((muon_m['iso']<0.25)&(muon_m.pt>4)&(np.abs(muon_m.eta)<2.8))]
        
mu_sel_t = (events.MuonTight.PT>4)
muon_t = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.MuonTight.PT[mu_sel_t],
    eta = events.MuonTight.Eta[mu_sel_t],
    phi = events.MuonTight.Phi[mu_sel_t],
    M = np.zeros_like(events.MuonTight.PT[mu_sel_t]),
    copy = False,
)
muon_t['iso'] = events.MuonTight.IsolationVar[mu_sel_t]

muon_t = muon_t[((muon_t['iso']<0.25)&(muon_t.pt>4)&(np.abs(muon_t.eta)<2.8))]
        
#taus
tau_sel = (events.JetPUPPI.PT>30)
tau = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.JetPUPPI.PT[tau_sel],
    eta = events.JetPUPPI.Eta[tau_sel],
    phi = events.JetPUPPI.Phi[tau_sel],
    M = events.JetPUPPI.Mass[tau_sel],
    copy = False,
)
tau['iso'] = events.JetPUPPI.TauTag[tau_sel]   # > 0 should be loose

tau_l = tau[((tau['iso']>0)&(tau.pt>30)&(np.abs(tau.eta)<3))]

#gen

gen_sel = ((abs(events.Particle.PID)==6) | (abs(events.Particle.PID)==5) | (abs(events.Particle.PID)==25))  # NOTE: attempt to speed up reading gigantic gen particle branches

gen = get_four_vec_fromPtEtaPhiM(
    None,
    pt = events.Particle.PT[gen_sel],
    eta = events.Particle.Eta[gen_sel],
    phi = events.Particle.Phi[gen_sel],
    M = events.Particle.Mass[gen_sel],
    copy = False,
)
gen['pdgId'] = events.Particle.PID[gen_sel]
gen['status'] = events.Particle.Status[gen_sel]

#higgs = gen[((abs(gen.pdgId)==25)&(gen.status==62))]
higgs = gen[(abs(gen.pdgId)==25)][:,-1:]  # just get the last Higgs. Delphes is not keeping all the higgses.

bquark = gen[((abs(gen.pdgId)==5)&(gen.status==71))]  # I suspect that Delphes does not keep b's with pt less than 20?

#top = gen[gen.pdgId==6][:,-2:-1]
#atop = gen[gen.pdgId==-6][:,-2:-1]
#ttbar = ak.flatten(cross(top, atop))

dibquark = choose(bquark, 2)
b_DeltaR = delta_r(dibquark['0'], dibquark['1'])


jet = getJets(events, jes_corrector, '', delphes=True)   #getJets uses JetPUPPILoose for delphes samples
            

#follow Delphes recommendations
jet = jet[jet.pt > 30]
jet = jet[np.abs(jet.eta) < 3] #eta within tracker range
jet = jet[~match(jet, ele_l, deltaRCut=0.4)] #remove electron overlap
jet = jet[~match(jet, ele_m, deltaRCut=0.4)]
jet = jet[~match(jet, ele_t, deltaRCut=0.4)]
jet = jet[~match(jet, muon_l, deltaRCut=0.4)] #remove muon overlap
jet = jet[~match(jet, muon_m, deltaRCut=0.4)]
jet = jet[~match(jet, muon_t, deltaRCut=0.4)]
jet = jet[ak.argsort(jet.pt, ascending=False)]

#fatjets

# Need FatJets and GenParts
# FatJets start at pt>200 and go all the way to eta 3.x
# This should be fine?
# Objects are defined here: https://twiki.cern.ch/twiki/bin/view/CMS/DelphesInstructions
# restrict abs(eta) to 2.8 (whatever the tracker acceptance of PhaseII CMS is)

               
fatjet = getFatjets(events, jes_corrector, '', '', delphes=True)

fatjet = fatjet[np.abs(fatjet.eta) < 3] #eta within tracker range        
fatjet = fatjet[~match(fatjet, ele_l, deltaRCut=0.8)] #remove electron overlap
fatjet = fatjet[~match(fatjet, ele_m, deltaRCut=0.8)] #remove electron overlap
fatjet = fatjet[~match(fatjet, ele_t, deltaRCut=0.8)] #remove electron overlap
fatjet = fatjet[~match(fatjet, muon_l, deltaRCut=0.8)] #remove muon overlap
fatjet = fatjet[~match(fatjet, muon_m, deltaRCut=0.8)] #remove muon overlap
fatjet = fatjet[~match(fatjet, muon_t, deltaRCut=0.8)] #remove muon overlap

fatjet_on_h = fatjet[np.abs(fatjet.mass-125)<25]
on_h = (ak.num(fatjet_on_h) > 0)

lead_fatjet = fatjet[:,0:1]
 
difatjet = choose(fatjet, 2)
dijet = choose(jet[:,:4], 2)  # only take the 4 leading jets

dphi_difatjet = np.arccos(np.cos(difatjet['0'].phi-difatjet['1'].phi))
dphi_dijet = np.arccos(np.cos(dijet['0'].phi-dijet['1'].phi))
AK8_QCD_veto = ak.all(dphi_difatjet<3.0, axis=1)  # veto any event with a back-to-back dijet system. No implicit cut on N_AK8 (ak.all!)
AK4_QCD_veto = ak.all(dphi_dijet<3.0, axis=1)  # veto any event with a back-to-back dijet system. No implicit cut on N_AK4 (ak.all!)

#MET

met_pt = ak.flatten(events.PuppiMissingET.MET)
genmet_pt = ak.flatten(events.GenMissingET.MET)
met_phi = ak.flatten(events.PuppiMissingET.Phi)

mt_AK8_MET = mt(fatjet.pt, fatjet.phi, met_pt, met_phi)
min_mt_AK8_MET = ak.to_numpy(ak.min(mt_AK8_MET, axis=1))
min_dphi_AK8_MET = ak.to_numpy(ak.min(np.arccos(np.cos(fatjet.phi-met_phi)), axis=1))
min_dphi_AK4_MET = ak.to_numpy(ak.min(np.arccos(np.cos(jet.phi-met_phi)), axis=1))

nb_in_fat = match_count(fatjet, bquark, deltaRCut=0.8)
nhiggs_in_fat = match_count(fatjet, higgs, deltaRCut=0.8)
zerohiggs = (nhiggs_in_fat==0)
onehiggs = (nhiggs_in_fat==1)

zerob = ((nb_in_fat==0) & (zerohiggs))  # verified to work!
oneb  = ((nb_in_fat==1) & (zerohiggs))  # verified to work!
twob  = ((nb_in_fat>=2) & (zerohiggs))  # verified to work!

w_0b = get_weight(effs['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['0b'], fatjet.pt, fatjet.eta)
w_1b = get_weight(effs['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['1b'], fatjet.pt, fatjet.eta)
w_2b = get_weight(effs['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['2b'], fatjet.pt, fatjet.eta)
w_1h = get_weight(effs['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['1h'], fatjet.pt, fatjet.eta)

In [ ]:
w_0b

In [ ]:
events_150 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_1500_150_10_delphes_ntuple.root:myana',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_250 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_1500_250_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_350 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_1500_350_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_500 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_1500_500_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()   

In [ ]:
events = NanoEventsFactory.from_root(
        '/hadoop/cms/store/user/ewallace/ProjectMetis/2HDMa_gg_1600_750_10_HLLHC_GEN_v5/delphes/2HDMa_gg_1600_750_10_delphes_ntuple.root:myana',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events() 

In [ ]:
events_1900_250 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_1900_250_10_delphes.root',
        treepath='',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
events_1900_250 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_1900_250_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
WJet = NanoEventsFactory.from_root(
        '/nfs-7/userdata/dspitzba/merge_WJetsToLNu_GenMET-100_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU_v16/merge_1.root',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
lheweights = WJet.lheweight_val
lheweight_ratio = {}
#for i in range(363,463):
#    lheweight_ratio[i] = lheweights[:,i]/lheweights[:,0]
#for i in [463,464]:
#    lheweight_ratio[i] = lheweights[:,i]/lheweights[:,0]
for i in [5, 10, 15, 20, 30, 40]:
    lheweight_ratio[i] = ak.flatten(lheweights[:,i:i+1]/lheweights[:,0:1])

In [ ]:
lheweight_ratio

In [ ]:
WJet.lheweight_size[WJet.lheweight_size==0]

In [ ]:
events_250 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_2250_250_10_delphes_ntuple.root:myana',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
len(events_250)

In [ ]:
zjets = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/merge_ZJetsToNuNu_HT-2500ToInf_14TeV-madgraph_200PU_v4/ZJetsToNuNu_HT2500toInf_delphes_ntuple.root:myana',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
test = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_bb_1000_500_10_delphes_ntuple.root:myana',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
len(test)

In [ ]:
fatjet_pt = zjets.fatjet_pt
jet_pt = zjets.jetpuppi_pt

In [ ]:
ak.sum(jet_pt, axis=1).tolist()

In [ ]:
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [ ]:
plt.hist(ak.sum(jet_pt, axis=1), bins=np.linspace(900,3900,12), histtype = 'step', label = 'ZJets2500toInf')
plt.xlabel(r'$H_T$')
plt.ylabel('Events')

In [ ]:
plt.hist(ak.flatten(fatjet_pt), bins=np.linspace(200,2000,10), histtype = 'step', label = 'ZJets2500toInf')

In [ ]:
from yaml import Loader, Dumper
import yaml
with open('../data/samples.yaml', 'r') as f:
    samples = yaml.load(f, Loader = Loader)

In [ ]:
samples.keys()

In [ ]:
samples['ST_tch_14TeV_top_incl-powheg-pythia8-madspin_200PU']['xsec']

In [ ]:
from yaml import Loader, Dumper
import yaml
    
with open('../data/samples.yaml', 'r') as f:
    samples = yaml.load(f, Loader = Loader)

fileset = {
            #'TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU': samples['TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU']['skim'],
            #'ZJetsToNuNu_HT-100To200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-100To200_14TeV-madgraph_200PU']['skim'],
            #'ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['skim'],
            #'ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU']['skim'],
            #'ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU']['skim'],
            #'ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU']['skim'],
            #'ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU']['skim'],
            #'W0JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W0JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'W1JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W1JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'W2JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W2JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'W3JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W3JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_350_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_350_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_500_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_500_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['ntuples'],
            '2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': samples['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['ntuples'],
}

meta_accumulator = {}
for sample in fileset:
    if sample not in meta_accumulator:
        meta_accumulator.update({sample: processor.defaultdict_accumulator(int)})
    for f in fileset[sample]:
        meta_accumulator.update({f: processor.defaultdict_accumulator(int)})

In [ ]:
events_250 = NanoEventsFactory.from_root(
        samples['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['ntuples'][0],
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
fatjet = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.fatjet_pt,
            eta = events_250.fatjet_eta,
            phi = events_250.fatjet_phi,
            M = events_250.fatjet_msoftdrop,        #Using softdrop from now on
            copy = False,
        )

In [ ]:
fatjet2 = getFatjets(events_250, jes_corrector,'',[0.05,0.1])

In [ ]:
fatjet.mass

In [ ]:
fatjet2.mass

In [ ]:
fatjet.pt[ak.num(fatjet)>1]

In [ ]:
electron = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.elec_pt,
            eta = events_250.elec_eta,
            phi = events_250.elec_phi,
            M = events_250.elec_mass,
            copy = False,
        )
electron['id'] = events_250.elec_idpass  # > 0 should be loose
electron['iso'] = events_250.elec_isopass
electron['charge'] = events_250.elec_charge

ele_l = electron[((electron['id']>0)&(electron['iso']>0)&(electron.pt>10)&(np.abs(electron.eta)<3))]
        
        #muons
        
muon = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.muon_pt,
            eta = events_250.muon_eta,
            phi = events_250.muon_phi,
            M = events_250.muon_mass,
            copy = False,
        )
muon['id'] = events_250.muon_idpass  # > 0 should be loose
muon['iso'] = events_250.muon_isopass
muon['charge'] = events_250.muon_charge

muon_l = muon[((muon['id']>0)&(muon['iso']>0)&(muon.pt>4)&(np.abs(muon.eta)<2.8))]
        
#taus
        
tau = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.tau_pt,
            eta = events_250.tau_eta,
            phi = events_250.tau_phi,
            M = events_250.tau_mass,
            copy = False,
        )
tau['iso'] = events_250.tau_isopass   # > 0 should be loose
tau['charge'] = events_250.tau_charge

tau_l = tau[((tau['iso']>0)&(tau.pt>30)&(np.abs(tau.eta)<3))]
        
#photons
        
gamma = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.gamma_pt,
            eta = events_250.gamma_eta,
            phi = events_250.gamma_phi,
            M = events_250.gamma_mass,
            copy = False,
        )
gamma['id'] = events_250.gamma_idpass  # > 0 should be loose
gamma['iso'] = events_250.gamma_isopass

gamma_l = gamma[((gamma['id']>0)&(gamma['iso']>0)&(gamma.pt>20)&(np.abs(gamma.eta)<3))]


jet = getJets(events_250, jes_corrector, pt_var='')
#jet['id'] = events_250.jetpuppi_idpass
#jet['btag'] = events_250.jetpuppi_btag
        
        
#follow Delphes recommendations
jet = jet[jet.pt > 30]
jet = jet[jet.id > 0]
jet = jet[np.abs(jet.eta) < 3] #eta within tracker range
        
jet = jet[~match(jet, ele_l, deltaRCut=0.4)] #remove electron overlap
jet = jet[~match(jet, muon_l, deltaRCut=0.4)] #remove muon overlap

btag = jet[jet.btag>0] #loose wp for now
        
#fatjets

# Need FatJets and GenParts
# FatJets start at pt>200 and go all the way to eta 3.x
# This should be fine?
# Objects are defined here: https://twiki.cern.ch/twiki/bin/view/CMS/DelphesInstructions
# Maybe restrict abs(eta) to 2.8 or 3 (whatever the tracker acceptance of PhaseII CMS is)
fatjet = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.fatjet_pt,
            eta = events_250.fatjet_eta,
            phi = events_250.fatjet_phi,
            M = events_250.fatjet_msoftdrop,        #Using softdrop from now on
            copy = False,
        )
        
#fatjet['m'] = events.fatjet_mass
fatjet['tau1'] = events_250.fatjet_tau1
fatjet['tau2'] = events_250.fatjet_tau2
fatjet['tau3'] = events_250.fatjet_tau3
fatjet['tau4'] = events_250.fatjet_tau4
        
fatjet = fatjet[np.abs(fatjet.eta) < 3] #eta within tracker range
fatjet = fatjet[ak.argsort(fatjet.pt, ascending=False)]
#
fatjet = fatjet[~match(fatjet, ele_l, deltaRCut=0.8)] #remove electron overlap
fatjet = fatjet[~match(fatjet, muon_l, deltaRCut=0.8)] #remove muon overlap
#
extrajet  = jet[~match(jet, fatjet, deltaRCut=1.2)] # remove AK4 jets that overlap with AK8 jets
extrabtag = extrajet[extrajet.btag>0] #loose wp for now]
        
tau21 = np.divide(fatjet.tau2, fatjet.tau1)
        
fatjet_on_h = fatjet[np.abs(fatjet.mass-125)<25]
on_h = (ak.num(fatjet_on_h) > 0)
        
lead_fatjet = fatjet[:,0:1]

difatjet = choose(fatjet, 2)
dijet = choose(jet[:,:4], 2)  # only take the 4 leading jets
di_AK8_AK4 = cross(extrajet, fatjet)

dphi_difatjet = np.arccos(np.cos(difatjet['0'].phi-difatjet['1'].phi))
dphi_dijet = np.arccos(np.cos(dijet['0'].phi-dijet['1'].phi))
dphi_AK8_AK4 = np.arccos(np.cos(di_AK8_AK4['0'].phi-di_AK8_AK4['1'].phi))
# not back-to-back
AK8_QCD_veto = ak.all(dphi_difatjet<3.0, axis=1)  # veto any event with a back-to-back dijet system. No implicit cut on N_AK8 (ak.all!)
AK4_QCD_veto = ak.all(dphi_dijet<3.0, axis=1)  # veto any event with a back-to-back dijet system. No implicit cut on N_AK4 (ak.all!)
min_dphi_AK8_AK4 = ak.to_numpy(ak.min(dphi_AK8_AK4, axis=1))

In [ ]:
met_pt = ak.flatten(events_250.metpuppi_pt)
met_phi = ak.flatten(events_250.metpuppi_phi)
        
mt_AK8_MET = mt(fatjet.pt, fatjet.phi, met_pt, met_phi)
min_mt_AK8_MET = ak.to_numpy(ak.min(mt_AK8_MET, axis=1))
min_dphi_AK8_MET = ak.to_numpy(ak.min(np.arccos(np.cos(fatjet.phi-met_phi)), axis=1))
min_dphi_AK4_MET = ak.to_numpy(ak.min(np.arccos(np.cos(jet.phi-met_phi)), axis=1))
min_dphi_AK4clean_MET = ak.to_numpy(ak.min(np.arccos(np.cos(extrajet.phi-met_phi)), axis=1))  

In [ ]:
run2_to_delphes = {
    #'ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8',
    #'ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8',
    #'ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-600To800_TuneCP5_13TeV-madgraphMLM-pythia8',
    #'ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8',
    #'ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-1200To2500_TuneCP5_13TeV-madgraphMLM-pythia8',
    '2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': 'ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8',
}

effs = {}
for s in fileset.keys():
    effs[s] = {}
    for b in ['0b', '1b', '2b', '1h']:
        effs[s][b] = Hist2D.from_json(os.path.expandvars("../data/htag/eff_%s_%s.json"%(run2_to_delphes[s],b)))

In [ ]:
gen = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.genpart_pt,
            eta = events_250.genpart_eta,
            phi = events_250.genpart_phi,
            M = events_250.genpart_mass,
            copy = False,
        )
gen['pdgId'] = events_250.genpart_pid
gen['status'] = events_250.genpart_status

higgs = gen[((abs(gen.pdgId)==25)&(gen.status==62))]

bquark = gen[((abs(gen.pdgId)==5)&(gen.status==71))]
        
nb_in_fat = match_count(fatjet, bquark, deltaRCut=0.8)
nhiggs_in_fat = match_count(fatjet, higgs, deltaRCut=0.8)
zerohiggs = (nhiggs_in_fat==0)
onehiggs = (nhiggs_in_fat==1)
        
zerob = ((nb_in_fat==0) & (zerohiggs))  # verified to work!
oneb  = ((nb_in_fat==1) & (zerohiggs))  # verified to work!
twob  = ((nb_in_fat>=2) & (zerohiggs))  # verified to work!
        
w_0b = get_weight(effs['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['0b'], fatjet.pt, fatjet.eta)
w_1b = get_weight(effs['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['1b'], fatjet.pt, fatjet.eta)
w_2b = get_weight(effs['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['2b'], fatjet.pt, fatjet.eta)
w_1h = get_weight(effs['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['1h'], fatjet.pt, fatjet.eta)
        
w_all = w_0b * zerob + w_1b * oneb + w_2b * twob # + w_1h * onehiggs  # this should work
if not np.isnan(sum(sum(effs['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['1h'].counts))):
#        np.isnan(sum(ak.flatten(w_1h * onehiggs))):
    w_all = w_all + w_1h * onehiggs

In [ ]:
ak.num(higgs)

In [ ]:
dibquark = choose(bquark, 2)

In [ ]:
dibquark

In [ ]:
b_DeltaR = delta_r(dibquark['0'], dibquark['1'])

In [ ]:
ak.flatten(b_DeltaR[ak.num(bquark)==2])

In [ ]:
pad_and_flatten(higgs.pt[ak.num(bquark)==2])

In [ ]:
weight = Weights(len(events_250))
weight.add("NH>0", np.nan_to_num(1-ak.prod(1-w_all, axis=1), 0))

In [ ]:
selection = PackedSelection()

selection.add('ele_veto', ak.num(ele_l, axis=1)==0)
selection.add('mu_veto',  ak.num(muon_l, axis=1)==0)
selection.add('tau_veto', ak.num(tau_l, axis=1)==0)
selection.add('met',      met_pt>300)
selection.add('nAK4',     ak.num(jet, axis=1)>1)
selection.add('nAK8',     ak.num(fatjet, axis=1)>0)
selection.add('min_AK8_pt', ak.min(fatjet.pt, axis=1)>300)
selection.add('dphi_AK8_MET>1', min_dphi_AK8_MET>1.0)
selection.add('dphi_AK4_MET<3', min_dphi_AK4_MET<3.0)
selection.add('dphi_AK4_MET>1', min_dphi_AK4_MET>1.0)
selection.add('AK4_QCD_veto', AK4_QCD_veto)
selection.add('AK8_QCD_veto', AK8_QCD_veto)
selection.add('on_H',     on_h)
selection.add('MT>600',   min_mt_AK8_MET>600)

In [ ]:
def n_minus_one(selection, requirements, minus_one):
    reqs_d = { sel: True for sel in requirements if not sel in minus_one }
    return selection.require(**reqs_d)

baseline = [
    'ele_veto',
    'mu_veto',
    'tau_veto',
    'met',
    'nAK8',
]

tight = [
    'ele_veto',
    'mu_veto',
'tau_veto',
    'met',
    'nAK8',
    'nAK4',
    'min_AK8_pt',
    'dphi_AK8_MET>1',
    'dphi_AK4_MET<3',
    'dphi_AK4_MET>1',
    'AK4_QCD_veto',
    'AK8_QCD_veto',
    'on_H',
    'MT>600',
]
        
base_sel = n_minus_one(selection, baseline, [])
tight_sel = n_minus_one(selection, tight, [])

In [ ]:
len(events_250[n_minus_one(selection, baseline, ['met', 'nAK8'])])

In [ ]:
len(events_250[n_minus_one(selection, baseline, ['ele veto', 'muon veto', 'tau veto', 'met', 'nAK8'])])

In [ ]:
len(events_250[n_minus_one(selection, baseline, ['nAK8'])])

In [ ]:
len(events_250[base_sel])

In [ ]:
dphi_difatjet[baseline &(ak.num(fatjet)>1)].tolist()

In [ ]:
weight.weight()[baseline &(ak.num(fatjet)>1)]

In [ ]:
ak.min(fatjet.pt[baseline], axis=1)

In [ ]:
events_QCD = NanoEventsFactory.from_root(
        '/hadoop/cms/store/user/dspitzba/ProjectMetis/merge_QCD_bEnriched_HT1000to1500_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU_v15/merge_1.root:nevents',
        #treepath='nevents',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
events_QCD

In [ ]:
import uproot

In [ ]:
with uproot.open('/hadoop/cms/store/user/ewallace/ProjectMetis/2HDMa_bb_1500_250_10_HLLHC_GEN_v4/gen_10.root:Events',) as events:
    print(events.keys())

In [ ]:
fileset = {
            'WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['WJetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
}

In [ ]:
with uproot.open('/hadoop/cms/store/user/dspitzba/ProjectMetis/merge_VVTo2L2Nu_14TeV_amcatnloFXFX_madspin_pythia8_200PU_v16/merge_2.root:nevents') as counts:
    print(counts.counts())

In [ ]:
nevents = {}

for key in fileset:
    nevents[key] = 0
    for file in fileset[key]:
        with uproot.open(file+':nevents') as counts:
            nevents[file] = counts.counts()[0] 
        nevents[key] += nevents[file]

In [ ]:
nevents